In [1]:
import pandas as pd
from constants import DATA_DIR, HEADER_TOTAL_EXPOSURE
# DATA_DIR = 'testA/'

In [2]:
total_exposure = pd.read_csv(DATA_DIR + 'totalExposureLog.out', sep='\t', names=HEADER_TOTAL_EXPOSURE,
                             header=None, index_col=False,dtype=str, quotechar='\t',
                             usecols=['req_id', 'req_ts', 'ad_loc_id', 'uid', 'ad_id', 'expo_size'])

In [3]:
#  如果是同一请求同一广告位同一广告多次曝光建议去重
print(len(total_exposure))
total_exposure.drop_duplicates(subset=['req_id', 'ad_loc_id', 'ad_id'], inplace=True)
print(len(total_exposure))
total_exposure.head()

102386695
101507776


,req_id,req_ts,ad_loc_id,uid,ad_id,expo_size
0,53991770,1550409746,94,1160618,451525,50
1,25942318,1550370892,79,203814,214797,64
2,66156247,1550416600,18,808543,92253,40
3,5935886,1550365898,198,7270,160082,64
4,11624425,1550361159,168,852707,253902,64


In [4]:
from datetime import timedelta
utc2bj = timedelta(hours=8)
total_exposure['date'] = (pd.to_datetime(total_exposure.req_ts, unit='s') + utc2bj).dt.strftime('%Y%m%d').astype(int)
total_exposure.drop('req_ts', axis=1, inplace=True)

In [5]:
ad_daily_exposure = total_exposure[['ad_id', 'date', 'req_id']].groupby(['ad_id', 'date']).agg('count')

In [6]:
ad_daily_exposure.rename(columns={'req_id': 'cnt'}, inplace=True)

In [7]:
print(len(ad_daily_exposure))
del(total_exposure)
print(len(ad_daily_exposure))

1913516
1913516


In [8]:
ad_daily_exposure[['ad_id', 'date']] = ad_daily_exposure.index.to_frame()
ad_daily_exposure.reset_index(drop=True, inplace=True)
ad_daily_exposure.head()

,cnt,ad_id,date
0,7,1,20190220
1,8,1,20190221
2,12,1,20190222
3,10,100,20190224
4,35,100,20190225


In [9]:
ad_daily_exposure[['ad_id', 'date', 'cnt']].to_csv(DATA_DIR + 'label.tsv', sep='\t', index=False)